# Querying SSISDB for information

Using the queries from https://github.com/yorek/ssis-queries to query the SSIS database (hopefully) quicker than the execution reports

Show the latest executed packages

In [ ]:
/*
	:: PURPOSE
	Show the latest executed packages
	
	:: NOTES
	For the *first* package in the list, also show
	. Performance of last 15 successful executions
	. Error messages
	. Duplicate lookup messages
	. Memory allocation warnings
	. Low virtual memory warnings
	
	:: INFO
	Author:		Davide Mauri
	Version:	1.0
*/
USE SSISDB
GO

/*
	Configuration
*/

-- Filter data by project name (use % for no filter)
DECLARE @projectNamePattern NVARCHAR(100) = 'TPMMRT'

-- Filter data by package name (use % for no filter)
DECLARE @packageNamePattern NVARCHAR(100) = '%'

-- Filter data by execution id (use NULL for no filter)
DECLARE @executionIdFilter BIGINT = NULL

/*
	Implementation
*/

-- Show last 15 executions
SELECT TOP 15
	e.execution_id, 
	e.project_name,
	e.package_name,
	e.project_lsn,
	e.status, 
	status_desc = CASE e.status 
						WHEN 1 THEN 'Created'
						WHEN 2 THEN 'Running'
						WHEN 3 THEN 'Cancelled'
						WHEN 4 THEN 'Failed'
						WHEN 5 THEN 'Pending'
						WHEN 6 THEN 'Ended Unexpectedly'
						WHEN 7 THEN 'Succeeded'
						WHEN 8 THEN 'Stopping'
						WHEN 9 THEN 'Completed'
					END,
	e.start_time,
	e.end_time,
	elapsed_time_min = datediff(mi, e.start_time, e.end_time)
FROM 
	catalog.executions e 
WHERE 
	e.project_name LIKE @projectNamePattern
AND
	e.package_name LIKE @packageNamePattern
AND
	e.execution_id = ISNULL(@executionIdFilter, e.execution_id)
ORDER BY 
	e.execution_id DESC
OPTION
	(RECOMPILE)
;


 Show successful execution history for last 15 runs

In [ ]:

-- Get detailed information for the first package in the list
USE SSISDB
GO

-- Filter data by project name (use % for no filter)
DECLARE @projectNamePattern NVARCHAR(100) = '%'

-- Filter data by package name (use % for no filter)
DECLARE @packageNamePattern NVARCHAR(100) = '%'

-- Filter data by execution id (use NULL for no filter)
DECLARE @executionIdFilter BIGINT = NULL

DECLARE @executionId BIGINT, @packageName NVARCHAR(1000) 
SELECT 
	TOP 1 @executionId = e.execution_id, @packageName = e.package_name 
FROM 
	[catalog].executions e
WHERE 
	e.project_name LIKE @projectNamePattern
AND
	e.package_name LIKE @packageNamePattern
AND
	e.execution_id = ISNULL(@executionIdFilter, e.execution_id)
ORDER BY 
	e.execution_id DESC
OPTION
	(RECOMPILE);

    
-- Show successfull execution history
SELECT TOP 15
e.execution_id, 
	e.project_name,
	e.package_name,
	e.project_lsn,
	e.status, 
	status_desc = CASE e.status 
						WHEN 1 THEN 'Created'
						WHEN 2 THEN 'Running'
						WHEN 3 THEN 'Cancelled'
						WHEN 4 THEN 'Failed'
						WHEN 5 THEN 'Pending'
						WHEN 6 THEN 'Ended Unexpectedly'
						WHEN 7 THEN 'Succeeded'
						WHEN 8 THEN 'Stopping'
						WHEN 9 THEN 'Completed'
					END,
	e.start_time,
	e.end_time,
	elapsed_time_min = datediff(mi, e.start_time, e.end_time)
FROM 
	catalog.executions e 
WHERE 
	e.status IN (2,7)
AND
	e.package_name = @packageName
ORDER BY 
	e.execution_id DESC
;


Get error information for the latest runs

In [ ]:

-- Get detailed information for the first package in the list
USE SSISDB
GO

-- Filter data by project name (use % for no filter)
DECLARE @projectNamePattern NVARCHAR(100) = '%'

-- Filter data by package name (use % for no filter)
DECLARE @packageNamePattern NVARCHAR(100) = '%'

-- Filter data by execution id (use NULL for no filter)
DECLARE @executionIdFilter BIGINT = NULL

DECLARE @executionId BIGINT, @packageName NVARCHAR(1000) 

SELECT 
	TOP 1 @executionId = e.execution_id, @packageName = e.package_name 
FROM 
	[catalog].executions e
WHERE 
	e.project_name LIKE @projectNamePattern
AND
	e.package_name LIKE @packageNamePattern
AND
	e.execution_id = ISNULL(@executionIdFilter, e.execution_id)
ORDER BY 
	e.execution_id DESC
OPTION
	(RECOMPILE);


-- Show error messages
PRINT 'Error Messages'
SELECT 
	* 
FROM 
	catalog.event_messages em 
WHERE 
	em.operation_id = @executionId
AND 
	em.event_name = 'OnError'
ORDER BY 
	em.event_message_id DESC
;

PRINT 'Duplicate Indexes'
-- Show warnings for duplicate lookups
SELECT 
	* 
FROM 
	catalog.event_messages em 
WHERE 
	em.operation_id = @executionId
AND 
	em.event_name = 'OnWarning' 
AND 
	message LIKE '%duplicate%' 
ORDER BY 
	em.event_message_id DESC
;

PRINT 'Memory Allocation Warnings'
-- Show warnings for memory allocations
SELECT 
	* 
FROM 
	catalog.event_messages em 
WHERE 
	em.operation_id = @executionId
AND 
	em.event_name = 'OnInformation' 
AND 
	message LIKE '%memory allocation%' 
ORDER BY 
	em.event_message_id DESC
;

PRINT 'Low Virtual Memory Warnings'
-- Show warnings for low virtual memory
SELECT 
	* 
FROM 
	catalog.event_messages em 
WHERE 
	em.operation_id = @executionId
AND 
	em.event_name = 'OnInformation' 
AND 
	message LIKE '%low on virtual memory%' 
ORDER BY 
	em.event_message_id DESC
;

Get the parameter values used for a specific run  

You will need to grab the execution id from a run above and edit the query

In [ ]:
/*
	:: PURPOSE
	Show the parameters values used for a specific execution
	
	:: NOTES
	The first resultset shows the values set via "parameters", the second via the "set" option
	
	:: INFO
	Author:		Davide Mauri
	Version:	1.0
*/
USE SSISDB
GO

/*
	Configuration
*/

-- Filter data by execution id 
DECLARE @executionIdFilter BIGINT = 46745 -- Add the execution id here

SELECT * FROM [catalog].[execution_parameter_values] WHERE [execution_id] = @executionIdFilter and [value_set] = 1

SELECT * FROM [catalog].[execution_property_override_values] WHERE [execution_id] = @executionIdFilter

Requires execution id setting
PURPOSE  
Show the Data Flow information of a specific Execution
	
NOTES  
The first resultset shows the packages and the second one shows the dataflows found in the packages, along with loaded number of rows set to the destination

In [ ]:
/*
	:: PURPOSE
	Show the Data Flow information of a specific Execution
	
	:: NOTES
	The first resultset shows the packages and the second one shows the 
	dataflows found in the packages, along with loaded number of rows
	set to the destination
	
	:: INFO
	Author:		Davide Mauri
	Version:	1.0

*/
USE SSISDB
GO

/*
	Configuration
*/

-- Filter data by execution id (use NULL for no filter)
DECLARE @executionIdFilter BIGINT = 46745;



/*
	Implementation
*/

WITH 
ctePRE AS 
(
	SELECT * FROM catalog.event_messages em 
	WHERE em.event_name IN ('OnPreExecute')
	
), 
ctePOST AS 
(
	SELECT * FROM catalog.event_messages em 
	WHERE em.event_name IN ('OnPostExecute')
)
SELECT
	b.operation_id,
	b.event_message_id,
	b.package_path,
	b.message_source_name,
	pre_message_time = b.message_time,
	post_message_time = e.message_time,
	duration_minutes = DATEDIFF(mi, b.message_time, e.message_time)
FROM
	ctePRE b
LEFT OUTER JOIN
	ctePOST e ON b.operation_id = e.operation_id AND b.package_name = e.package_name AND b.message_source_id = e.message_source_id
WHERE
	b.operation_id = @executionIdFilter
AND
	b.package_path = '\Package'
ORDER BY
	b.event_message_id desc
;

WITH cte AS
(
	SELECT
		*,
		token_destination_name_start = CHARINDEX(': "', [message]) + 3,
		token_destination_name_end = CHARINDEX('" wrote', [message]),
		token_rows_start = LEN([message]) - CHARINDEX('e', REVERSE([message]), 1) + 3,
		token_rows_end = LEN([message]) - CHARINDEX('r', REVERSE([message]), 1)
	FROM
		[catalog].[event_messages] em
)
SELECT TOP 100
	event_message_id,
	package_name,
	message_source_name,
	message_time,
	destination_name = SUBSTRING([message], token_destination_name_start,  token_destination_name_end - token_destination_name_start),
	loaded_rows = SUBSTRING([message], token_rows_start, token_rows_end - token_rows_start),
	[message]
FROM 
	cte as c 
WHERE
	c.operation_id = @executionIdFilter
AND 
	subcomponent_name = 'SSIS.Pipeline' 
AND 
	[message] like '%rows.%'
ORDER BY 
	event_message_id DESC